# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Para Aplicações de IA com PyTorch e Lightning</font>

## <font color='blue'>Mini-Projeto 1</font>
## <font color='blue'>Aplicação de IA Para Análise de Sentimento de Textos em Português</font>

![DSA](imagens/MP1.png)

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [3]:
!pip install -q torch==2.0.0

In [4]:
!pip install -q transformers==4.28.1

In [5]:
# Imports
import csv
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [6]:
from transformers import logging
logging.set_verbosity_error()

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

torch       : 2.0.0
transformers: 4.28.1
csv         : 1.0



## Carregando os Dados

In [8]:
# Caminho do arquivo CSV que você deseja ler
csv_file_path = 'dados/frases.csv'

In [9]:
# Crie uma lista vazia para armazenar as frases
frases = []

In [10]:
# Abra o arquivo CSV no modo leitura ('r') e use o objeto 'csv.reader' para ler o arquivo
with open(csv_file_path, 'r', encoding='utf-8') as file:
    
    csv_reader = csv.reader(file)

    # Iterar sobre cada linha no arquivo CSV
    for row in csv_reader:
        frase = row[0]
        frases.append(frase)

In [11]:
# Imprime a lista de frases
print(frases)

['Eu amei este filme!', 'Este produto é terrível.', 'A comida estava deliciosa.', 'O serviço foi péssimo.', 'Este livro é incrível.', 'A história da Inglaterra é curiosa.', 'Não gostei daquele novo modelo de carro.', 'Aqueles foi um dos melhores jogos de futebol da história.', 'Este curso é fantástico e de alto nível.', 'O serviço do garçom não esteve à altura do nome do restaurante.']


In [12]:
# Nome do objeto 
texts = frases

In [13]:
# 1: positivo, 0: negativo
labels = [1, 0, 1, 0, 1, 1, 0, 1, 1, 0]  

## Pré-Processamento

In [14]:
RANDOM_SEED = 42

In [15]:
# Divisão dos dados em treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, 
                                                                      labels, 
                                                                      test_size = 0.2, 
                                                                      random_state = RANDOM_SEED)

In [16]:
# Carrega o tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
# Classe de tokenização dos dados
class SentimentAnalysisTokenizer(Dataset):
    
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer.encode_plus(text,
                                            add_special_tokens = True,
                                            max_length = self.max_length,
                                            padding = 'max_length',
                                            truncation = True,
                                            return_tensors = 'pt')

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label)
        }

In [18]:
MAX_LENGTH = 64

In [19]:
# Aplica a tokenização
train_dataset = SentimentAnalysisTokenizer(train_texts, train_labels, tokenizer, MAX_LENGTH)

In [20]:
# Aplica a tokenização
test_dataset = SentimentAnalysisTokenizer(test_texts, test_labels, tokenizer, MAX_LENGTH)

In [21]:
# Cria o data loader
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)

In [22]:
# Cria o data loader
test_loader = DataLoader(test_dataset, batch_size = 16)

## Loop de Treino, Avaliação e Inferência

In [23]:
# Função para treinar o modelo
def train_epoch(model, data_loader, criterion, optimizer, device):
    
    model.train()
    total_loss = 0

    for batch in data_loader:
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)

In [24]:
# Função para avaliar o modelo
def eval_epoch(model, data_loader, criterion, device):
    
    model.eval()
    total_loss = 0

    with torch.no_grad():
        
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(data_loader)

In [25]:
# Função para obter previsões do modelo
def predict(model, data_loader, device):
    
    model.eval()
    predictions = []

    with torch.no_grad():
        
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.tolist())

    return predictions

## Construção do Modelo

In [26]:
# Inicialização do modelo e configuração do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
# Carrega o modelo
modelo = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

In [28]:
# Coloca o modelo na memória do device
modelo.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [29]:
# Hiperparâmetros
EPOCHS = 10
LEARNING_RATE = 2e-5

In [30]:
# Otimizador
optimizer = torch.optim.AdamW(modelo.parameters(), lr = LEARNING_RATE)

**torch.optim.AdamW()** é uma classe de otimizador no PyTorch que implementa a versão AdamW do algoritmo de otimização Adam. AdamW é uma extensão do otimizador Adam que inclui correções de decaimento de peso (Weight Decay) para melhorar a generalização do modelo em tarefas de aprendizado profundo.

Aqui está uma descrição dos principais parâmetros da classe torch.optim.AdamW:

- params (iterável): Um iterável de tensores (geralmente os parâmetros do modelo) que serão otimizados.


- lr (float, opcional): Taxa de aprendizado (learning rate), um hiperparâmetro que controla o tamanho do passo de atualização dos parâmetros do modelo. O valor padrão é 1e-3 (0.001).


- betas (par de floats, opcional): Coeficientes usados para calcular as médias móveis exponenciais dos gradientes e seus quadrados. Os valores padrão são (0.9, 0.999).


- eps (float, opcional): Termo adicionado para melhorar a estabilidade numérica durante a otimização. O valor padrão é 1e-8.


-  weight_decay (float, opcional): Coeficiente de decaimento de peso (Weight Decay), um hiperparâmetro que penaliza os parâmetros do modelo para evitar overfitting. O valor padrão é 0.


- amsgrad (bool, opcional): Se True, usa a variante AMSGrad do otimizador Adam, que é mais robusta em certos casos, mas geralmente não é necessário. O valor padrão é False.

In [31]:
# Função de perda
criterion = torch.nn.CrossEntropyLoss()

**torch.nn.CrossEntropyLoss()** é uma classe de função de perda (loss function) no PyTorch, usada para tarefas de classificação multi-classe. A função de perda de entropia cruzada combina a função LogSoftmax e a função de perda NLLLoss (Negative Log-Likelihood Loss) em uma única classe, tornando-a conveniente e eficiente para uso em problemas de classificação.

A entropia cruzada mede a diferença entre duas distribuições de probabilidade, neste caso, a distribuição prevista pelo modelo e a distribuição real (rótulos verdadeiros). Durante o treinamento, o objetivo é minimizar a perda de entropia cruzada, o que leva a um melhor ajuste do modelo aos dados.

## Treinamento e Avaliação do Modelo

In [32]:
# Treinamento e validação do modelo
for epoch in range(EPOCHS):
    train_loss = train_epoch(modelo, train_loader, criterion, optimizer, device)
    test_loss = eval_epoch(modelo, test_loader, criterion, device)
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss}, Test Loss: {test_loss}')

Epoch 1/10, Train Loss: 0.7264857888221741, Test Loss: 0.6710620522499084
Epoch 2/10, Train Loss: 0.7300401329994202, Test Loss: 0.6715528964996338
Epoch 3/10, Train Loss: 0.6676653623580933, Test Loss: 0.6787925958633423
Epoch 4/10, Train Loss: 0.6567515134811401, Test Loss: 0.689117968082428
Epoch 5/10, Train Loss: 0.6349284052848816, Test Loss: 0.7016345858573914
Epoch 6/10, Train Loss: 0.6053487658500671, Test Loss: 0.7095603346824646
Epoch 7/10, Train Loss: 0.552311897277832, Test Loss: 0.7187110781669617
Epoch 8/10, Train Loss: 0.49440866708755493, Test Loss: 0.7537600994110107
Epoch 9/10, Train Loss: 0.4578571319580078, Test Loss: 0.7944836616516113
Epoch 10/10, Train Loss: 0.4430603086948395, Test Loss: 0.786106288433075


In [33]:
# Salva o modelo em disco
torch.save(modelo, 'modelos/modelo_dsa_mp1.pt')

In [34]:
# Carrega o modelo do disco
modelo_final = torch.load('modelos/modelo_dsa_mp1.pt')

## Testando o Modelo com Novos Dados

In [35]:
# Novos dados
novas_frases = ['Eu gostei muito deste filme.',
                'O atendimento do restaurante foi decepcionante.']

In [36]:
# Aplica a tokenização
dataset = SentimentAnalysisTokenizer(novas_frases, [0] * len(novas_frases), tokenizer, MAX_LENGTH)

In [37]:
# Cria o data loader
loader = DataLoader(dataset, batch_size = 16)

In [38]:
# Previsões
previsoes = predict(modelo_final, loader, device)

In [39]:
# Análise de sentimento
for text, prediction in zip(novas_frases, previsoes):
    print(f'Sentença: {text} | Sentimento: {"positivo" if prediction else "negativo"}')

Sentença: Eu gostei muito deste filme. | Sentimento: positivo
Sentença: O atendimento do restaurante foi decepcionante. | Sentimento: negativo


# Fim